In [1]:
import pandas as pd

In [19]:
url = './data-maxtemp/timeseries-data/jma-maxtemp-heatpoints-by-pref-daily.csv'

In [20]:
df = pd.read_csv(url, parse_dates=['date'])

東京都の記録

In [21]:
tokyo = df.loc[df.pref=='東京都'].copy()

In [22]:
def count_consecutive_true(group):
    is_true = group['over30_capitol']
    group['consecutive_count'] = (is_true.groupby((is_true != is_true.shift()).cumsum()).cumcount() + 1) * is_true
    return group

tokyo = tokyo.groupby('year').apply(count_consecutive_true).reset_index(drop=True)

In [23]:
tokyo

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol,consecutive_count
0,2000-01-01,2000,東京都,24.1,12.6,False,False,False,False,False,False,0
1,2000-01-02,2000,東京都,24.1,8.8,False,False,False,False,False,False,0
2,2000-01-03,2000,東京都,23.8,16.5,False,False,False,False,False,False,0
3,2000-01-04,2000,東京都,24.1,14.0,False,False,False,False,False,False,0
4,2000-01-05,2000,東京都,24.2,11.6,False,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8625,2023-08-13,2023,東京都,33.9,32.9,True,True,False,False,False,False,39
8626,2023-08-14,2023,東京都,32.9,32.9,True,True,False,False,False,False,40
8627,2023-08-15,2023,東京都,32.8,32.8,True,True,False,False,False,False,41
8628,2023-08-16,2023,東京都,32.2,32.2,True,True,False,False,False,False,42


In [24]:
tokyo[(tokyo.year<2023)&(tokyo.consecutive_count>=40)]

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol,consecutive_count
1687,2004-08-14,2004,東京都,37.0,34.7,True,True,True,False,False,False,40


In [9]:
tokyo[(tokyo.year==2023)&(tokyo.consecutive_count>=40)]

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol,consecutive_count
8626,2023-08-14,2023,東京都,32.9,32.9,True,True,False,False,False,False,40
8627,2023-08-15,2023,東京都,32.8,32.8,True,True,False,False,False,False,41
8628,2023-08-16,2023,東京都,32.2,32.2,True,True,False,False,False,False,42


In [10]:
tokyo[tokyo.year==2023]

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol,consecutive_count
8401,2023-01-01,2023,東京都,24.4,13.0,False,False,False,False,False,False,0
8402,2023-01-02,2023,東京都,24.4,12.1,False,False,False,False,False,False,0
8403,2023-01-03,2023,東京都,24.4,11.0,False,False,False,False,False,False,0
8404,2023-01-04,2023,東京都,24.7,11.0,False,False,False,False,False,False,0
8405,2023-01-05,2023,東京都,24.8,10.6,False,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8625,2023-08-13,2023,東京都,33.9,32.9,True,True,False,False,False,False,39
8626,2023-08-14,2023,東京都,32.9,32.9,True,True,False,False,False,False,40
8627,2023-08-15,2023,東京都,32.8,32.8,True,True,False,False,False,False,41
8628,2023-08-16,2023,東京都,32.2,32.2,True,True,False,False,False,False,42


In [11]:
# # 東京の夏の最高気温（ヒートマップ作成用）
#旧04_tokyo_temperature_heatmap.py

#####過去データ#########################################
# - 1963〜2022年（6月1日〜9月31日）の東京（東京都）の気温(データソース：気象庁「過去の気象データ・ダウンロード」https://www.data.jma.go.jp/risk/obsdl/index.php)
# - 2023年は6月分のデータをダウンロードし、上記過去分と結合してcsvに（github_data/1963_2022_all_202306only.csv）
######################################################

#####最新データ#########################################
# - 2023年の7月以降はjma-maxtempから取得した県別データから東京都を抜粋
# - 出力データ（ヒートマップ用）：'tokyo_max_temp.csv'（東京過去70年分の最高気温）
######################################################

import pandas as pd
from datetime import datetime, date

# 過去データ
filepath = './data-tokyo/1963_2022_all_202306only.csv'
past_data = pd.read_csv(filepath, parse_dates =['date'])

#最新データ
filepath = './data-maxtemp/timeseries-data/jma-maxtemp-heatpoints-by-pref-daily.csv'
latest = pd.read_csv(filepath, parse_dates=['date'])
#必要箇所を抜粋
latest = latest.loc[(latest.pref=='東京都'),['date','maxtemp_capitol','year']]
latest = latest.rename(columns={'maxtemp_capitol':'max'})

#統合
tokyo = pd.concat([past_data, latest])
#重複削除
tokyo = tokyo[~tokyo.duplicated(subset=['date'], keep='last')]

#日付
#（不要分もあるがあとで整理）
#念のため抜け漏れを埋める
dates_ = pd.date_range(tokyo.date.min(), tokyo.date.max())
dates_ = [d for d in dates_ if (d.month>5) & (d.month<10)]
tokyo = tokyo.set_index('date').reindex(dates_).rename_axis('date').reset_index()
#フォーマット
tokyo.month = tokyo.date.dt.month
tokyo.day = tokyo.date.dt.day
tokyo.month_day = tokyo.date.dt.strftime('%m-%d')
tokyo['month_day_flourish'] = tokyo.date.dt.strftime('%-m/%-d')

# 保存
#tokyo.to_csv('./data-tokyo/tokyo_max_temp.csv', index=False)

In [12]:
latest

,date,max,year
11,2000-01-01,12.6,2000
58,2000-01-02,8.8,2000
105,2000-01-03,16.5,2000
152,2000-01-04,14.0,2000
199,2000-01-05,11.6,2000
...,...,...,...
405401,2023-08-13,32.9,2023
405448,2023-08-14,32.9,2023
405495,2023-08-15,32.8,2023
405542,2023-08-16,32.2,2023


In [13]:
tokyo

,date,max,year,month,day,month_day,month_day_flourish
0,1963-06-01,27.7,1963,6,1,06-01,6/1
1,1963-06-02,25.6,1963,6,2,06-02,6/2
2,1963-06-03,19.7,1963,6,3,06-03,6/3
3,1963-06-04,16.7,1963,6,4,06-04,6/4
4,1963-06-05,22.0,1963,6,5,06-05,6/5
...,...,...,...,...,...,...,...
7393,2023-08-13,32.9,2023,8,13,08-13,8/13
7394,2023-08-14,32.9,2023,8,14,08-14,8/14
7395,2023-08-15,32.8,2023,8,15,08-15,8/15
7396,2023-08-16,32.2,2023,8,16,08-16,8/16


In [15]:
daily = pd.read_csv('https://raw.githubusercontent.com/Nikkei-Visual-Data-Journalism/Heatwave/main/data-maxtemp/timeseries-data/jma-maxtemp-heatpoints-by-pref-daily.csv')

In [18]:
daily[daily.pref=='東京都']

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol
11,2000-01-01,2000,東京都,24.1,12.6,False,False,False,False,False,False
58,2000-01-02,2000,東京都,24.1,8.8,False,False,False,False,False,False
105,2000-01-03,2000,東京都,23.8,16.5,False,False,False,False,False,False
152,2000-01-04,2000,東京都,24.1,14.0,False,False,False,False,False,False
199,2000-01-05,2000,東京都,24.2,11.6,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
405401,2023-08-13,2023,東京都,33.9,32.9,True,True,False,False,False,False
405448,2023-08-14,2023,東京都,32.9,32.9,True,True,False,False,False,False
405495,2023-08-15,2023,東京都,32.8,32.8,True,True,False,False,False,False
405542,2023-08-16,2023,東京都,32.2,32.2,True,True,False,False,False,False
